# **Programando un Encoder**

## Primera etapa: Bloque de Self-attention
Primero definimos un corpus de unas 5 frases y unas 10 palabras

In [1]:
corpus = [
    'El anime es un buen hobby',
    'El manga es más popular que los comics',
    'El anime es más popular que el manga',
    'La música es un buen hobby',
    'El anime es más popular que la música',
]

Definimos el vocabulario de nuestro corpus

In [2]:
vocab = set()

for frase in corpus:
    for palabra in frase.lower().split():
        vocab.add(palabra)
        
vocab = sorted(vocab)

print(vocab)
print(len(vocab))

['anime', 'buen', 'comics', 'el', 'es', 'hobby', 'la', 'los', 'manga', 'más', 'música', 'popular', 'que', 'un']
14


Vamos a transformar a vector las frases, creando primero una instancia de Vectorizer y luego aplicando fit_transform

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

x = vectorizer.fit_transform(corpus).toarray()

dk = x.shape[1]

print(x)

[[1 1 0 1 1 1 0 0 0 0 0 0 0 1]
 [0 0 1 1 1 0 0 1 1 1 0 1 1 0]
 [1 0 0 2 1 0 0 0 1 1 0 1 1 0]
 [0 1 0 0 1 1 1 0 0 0 1 0 0 1]
 [1 0 0 1 1 0 1 0 0 1 1 1 1 0]]


Creamos la matriz entrenable para Q, esto se define por numeros aleatorios usando rand(). Luego usamos producto punto entre x y wq para obtener Q, que será el primer parametro para calcular Attention(Q,K,V)

In [4]:
import numpy as np

wq = np.random.rand(dk, dk)

print(wq.shape)

q = np.dot(x, wq)

print(q.shape)

(14, 14)
(5, 14)


Hacemos el mismo procedimiento para K

In [5]:
wk = np.random.rand(dk, dk)

print(wq.shape)

k = np.dot(x, wk)

print(k.shape)

(14, 14)
(5, 14)


Finalmente el mismo proceso se aplica para obtener V

In [6]:
wv = np.random.rand(dk, dk)

print(wv.shape)

v = np.dot(x, wv)

print(v.shape)

(14, 14)
(5, 14)


Ahora según la definición de softmax, tenemos la siguiente fórmula:

$$
    softmax(z)_i = \frac{e^{z_i}}{\sum_{j=1}^n{e^{z_j}}}
$$

También la fórmula de Attention(Q,K,V) se define de la siguiente forma:

$$
Attention(Q,K,V) = softmax(\frac{Q.K_T}{sqrt{d_k}}).V
$$

In [7]:
scores = (np.dot(q, k.T) / np.sqrt(dk))

print(scores)

[[32.97126002 51.89504973 48.43355919 33.78079302 47.95604323]
 [42.61320446 67.28849646 63.50355035 43.80954286 62.28662937]
 [45.69686486 71.97222966 68.01230343 46.48062696 66.47584517]
 [30.56577016 48.51749389 44.99623333 31.50346291 44.43792596]
 [43.60056793 69.30609107 64.90946071 44.6324581  63.44948821]]


Definimos los attention weights para scores

In [9]:
exp_z = np.exp(scores)

aw = exp_z / np.sum(exp_z)

output = np.dot(aw, v)

Todo lo que hemos trabajado se resume en la función de Self-Attention

In [ ]:
def self_attention(Q, K, V):
    scores = np.dot(Q, K.T) / np.sqrt(Q.shape[1])
    attention_weights = np.exp(scores) / np.sum(np.exp(scores), axis=1)[:, None]
    return np.dot(attention_weights, V)

## Multi-head attention

